In [ ]:
# 06장 파일과 디렉터리 다루기
# -------------------------
# 프로그래밍 시 파일과 디렉터리 처리는 기본 중의 기본이다. 이번 장에서는 파일과 디렉터리를 다루는 모듈을 알아본다.

## 035 파일 경로를 객체로 다루려면? ― pathlib

In [ ]:
# 035 파일 경로를 객체로 다루려면? ― pathlib
# ----------------------------------------
# pathlib은 파일 시스템 경로를 문자열이 아닌 객체로 만들어 여러 가지 일을 할 수 있도록 하는 모듈이다.

# pathlib 모듈은 파이썬 3.4 버전부터 사용할 수 있다.

# 문제
# ----
# 다음은 현재 디렉터리의 모든 텍스트 파일(.txt)을 archive라는 디렉터리로 이동하는 일반적인 파이썬 코드이다
# (단, archive 디렉터리는 현재 디렉터리 하위에 이미 있다고 가정한다).

# import glob
# import os
# import shutil

# for file_path in glob.glob('%s/*.txt' % os.getcwd()):
#     parent = os.path.dirname(file_path)
#     filename = os.path.basename(file_path)
#     new_path = os.path.join(parent, 'archive', filename)
#     shutil.move(file_path, new_path)

# 이 코드를 glob, os, shutil 대신 pathlib만을 사용하도록 수정하려면 어떻게 해야 할까?


In [8]:
import glob
import os
import shutil

for file_path in glob.glob('%s/*.txt' % os.getcwd()):
    print(file_path)
    print('-'*70)
    
    parent = os.path.dirname(file_path)
    print(parent)
    print('-'*70)
    
    file_name = os.path.basename(file_path)
    print(file_name)
    print('-'*70)
    
    new_path = os.path.join(parent, 'archive', file_name)
    print(new_path)
    print('='*70)

    shutil.move(file_path, new_path)

In [ ]:
# 풀이
# ----
# 다음처럼 pathlib만을 사용하여 glob, os.path, shutil에서 사용했던 모든 기능을 구현할 수 있다.

# [파일명: pathlib_sample.py]

# import pathlib


# for p in pathlib.Path.cwd().glob('*.txt'):
#     new_p = p.parent.joinpath('archive', p.name)
#     p.replace(new_p)

# pathlib.Path.cwd()는 현재 디렉터리 객체(Path)를 반환한다. 
# 그리고 Path 객체의 glob() 함수로 해당 디렉터리에 있는 모든 txt 파일을 객체(Path 이터레이터)로 얻을 수 있다. 
# Path 객체의 parent()는 os.path.dirname()처럼 해당 파일의 현재 디렉터리를 반환한다. 
# 마찬가지로 Path 객체의 joinpath()는 os.path.join()처럼 파일 경로와 파일명을 합쳐 새로운 경로를 만든다. 
# 그리고 파일을 옮기고자 shutil.move() 대신 Path 객체의 replace() 함수를 사용했다.

# 하위 디렉터리까지 검색하려면 glob() 대신 rglob()을 사용하면 된다.

# pathlib를 사용하지 않은 코드와 사용한 코드를 비교하면 다음 표와 같다.

# 전통적인 방식	pathlib 방식(p는 pathlib로 생성한 Path 객체)
# os.getcwd	p.cwd
# glob.glob	p.glob
# os.path.dirname	p.parent
# os.path.basename	p.name
# os.path.join	p.joinpath
# shutil.move	p.replace


# 알아두면 좋아요
# 현재 디렉터리의 모든 파일을 조사하여 확장자별 개수 구하기
# -----------------------------------------------------
# >>> import collections, pathlib
# >>> collections.Counter([p.suffix for p in pathlib.Path.cwd().iterdir()])
# Counter({'.md': 2, '.txt': 4, '.pdf': 2, '.py': 1})

# iterdir()는 해당 디렉터리의 모든 파일을 이터레이터로 반환한다. Path 객체의 suffix는 .을 포함한 파일 확장자를 뜻한다.

# 참고 : 010 사용한 단어 개수를 구하려면? - collections.Counter

# 참고
# glob - 파일 검색: https://wikidocs.net/110619
# pathlib - 객체 지향 파일 시스템 경로: https://docs.python.org/ko/3/library/pathlib.html
# 하위 디렉터리 검색하기: https://wikidocs.net/39

In [24]:
import pathlib

for p in pathlib.Path.cwd().glob('*.txt'):
        new_p = p.parent.joinpath('archive', p.name)
        p.replace(new_p)

In [42]:
import pathlib, collections

print(collections.Counter([ p.suffix for p in pathlib.Path.cwd().iterdir() ]))
collections.Counter([ p.suffix for p in pathlib.Path.cwd().iterdir() ]).most_common()

Counter({'.ipynb': 7, '.txt': 4, '': 3})


[('.ipynb', 7), ('.txt', 4), ('', 3)]

## 036 디렉터리의 구성을 알려면? ― os.path

In [ ]:
# 036 디렉터리의 구성을 알려면? ― os.path
# -------------------------------------
# os.path는 경로명과 파일명에 대한 유용한 함수를 제공하는 모듈이다.

# 문제
# ----
# 특정 폴더(디렉터리)에 어떤 파이썬 파일이 있는지 알고자 한다. 
# 이럴 때 매개변수에 지정한 디렉터리의 파이썬 파일을 모두 출력하는 search(dirname) 함수는 어떻게 작성해야 할까? 
# 단, 하위 디렉터리의 파이썬 파일도 모두 출력해야 한다.


In [ ]:
# 풀이
# ----
# 디렉터리 경로를 입력으로 받아 파이썬 파일을 출력하는 search() 함수는 다음과 같다.

# [파일명: os_path_sample.py]

# import os

# def search(dirname):
#     filenames = os.listdir(dirname)
#     for filename rin filenames:
#         filepath = os.path.join(dirname, filename)
#         if os.path.isdir(filepath):
#             search(filepath)
#         elif os.path.isfile(filepath):
#             name, ext = os.path.splitext(filepath)
#             if ext == '.py': 
#                 print(filepath)

# search("c:/projects/pylib")  # c:/projects/pylib 디렉터리와 하위 디렉터리의 모든 .py 파일 출력
# 입력으로 받은 디렉터리의 모든 파일을 os.listdir()로 얻고 확장자가 .py이면 파일명을 출력한다. 
# os.listdir()로 얻은 경로가 파일이 아닌 디렉터리라면 다시 그 경로를 입력으로 하여 search() 함수를 재귀호출한다.

# 여기서 사용한 os 모듈과 os.path 모듈을 정리한 표는 다음과 같다.

# 사용 모듈	설명
# -------------
# os.listdir(path)	path 하위의 파일(디렉터리 포함)을 리스트로 반환한다.
# os.path.join(path, *paths)	path와 paths에 해당하는 모든 경로를 더하여 전체 경로를 반환한다. 
#                               이때 경로 구분자는 실행하는 운영체제에 따라 달라진다. (경로 구분자의 예: 유닉스는 /, 윈도우는 \\)
# os.path.isdir(path)	path의 디렉터리 여부를 반환한다.
# os.path.isfile(path)	path의 파일 여부를 반환한다.
# os.path.splitext(path)	path를 파일명과 확장자로 구분한다. (이때 확장자에는 마침표(.)를 포함한다.)


# 알아두면 좋아요
# pathlib를 사용하고 싶다면?
# -------------------------
# pathlib를 사용하여 만든 search() 함수는 다음과 같다.

# import pathlib

# def search(dirname):
#     for p in pathlib.Path(dirname).rglob('*.py'):
#         print(p)
# pathlib와 관련한 내용은 앞 절을 참고하자.

# 참고
# os.path - 일반적인 경로명 조작: https://docs.python.org/ko/3/library/os.path.html

In [49]:
import pathlib

def search(dirname):
    for p in pathlib.Path(dirname).rglob('*.ipynb'):
        print(p)
        
search('.')        

00. 들어가기 전에.ipynb
01. 텍스트 다루기.ipynb
02. 바이너리 데이터 다루기.ipynb
03. 다양한 데이터 다루기.ipynb
04. 수학과 숫자 다루기.ipynb
05. 함수형 프로그래밍 다루기.ipynb
06 . 파일과 디렉터리 다루기.ipynb
test_bed\03. 다양한 데이터 다루기.ipynb
test_bed\04. 수학과 숫자 다루기.ipynb
test_bed\05. 함수형 프로그래밍 다루기.ipynb
test_bed\06 . 파일과 디렉터리 다루기.ipynb


In [46]:
# import os

# def search(dirname):
#     filenames = os.listdir(dirname)
#     for filename rin filenames:
#         filepath = os.path.join(dirname, filename)
#         if os.path.isdir(filepath):
#             search(filepath)
#         elif os.path.isfile(filepath):
#             name, ext = os.path.splitext(filepath)
#             if ext == '.py': 
#                 print(filepath)

import os

def search(dirname):
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        if os.path.isdir(filepath):
            search(filepath)
        elif os.path.isfile(filepath):
            name, ext = os.path.splitext(filepath)
            if ext == '.ipynb':
                print(filepath)

search('.')            

.\00. 들어가기 전에.ipynb
.\01. 텍스트 다루기.ipynb
.\02. 바이너리 데이터 다루기.ipynb
.\03. 다양한 데이터 다루기.ipynb
.\04. 수학과 숫자 다루기.ipynb
.\05. 함수형 프로그래밍 다루기.ipynb
.\06 . 파일과 디렉터리 다루기.ipynb
.\test_bed\03. 다양한 데이터 다루기.ipynb
.\test_bed\04. 수학과 숫자 다루기.ipynb
.\test_bed\05. 함수형 프로그래밍 다루기.ipynb
.\test_bed\06 . 파일과 디렉터리 다루기.ipynb


## 037 여러 개의 파일을 한꺼번에 읽으려면? ― fileinput

In [ ]:
# 037 여러 개의 파일을 한꺼번에 읽으려면? ― fileinput
# -------------------------------------------------
# fileinput은 여러 개의 파일을 한꺼번에 처리할 때 사용하는 모듈이다.

# 문제
# ----
# 현재 디렉터리에 텍스트 파일(.txt)이 너무 많아 하나씩 열어 내용을 확인하려니 너무 번거롭다. 
# 이럴 때 모든 텍스트 파일을 읽어 그 내용을 한 줄씩 출력하려면 어떻게 코드를 작성해야 할까?


In [ ]:
# 풀이
# ----
# 여러 개의 파일을 한꺼번에 처리할 때는 다음처럼 fileinput 모듈을 사용하는 것이 가장 편리하다.

# [파일명: fileinput_sample.py]

# import fileinput
# import glob

# with fileinput.input(glob.glob("*.txt")) as f:
#     for line in f:
#         print(line)
# 파일 단위가 아니라 모든 txt 파일 전체를 하나의 파일처럼 처리하고자 fileinput.input(glob.glob("*.txt"))와 같이 사용했다.

# 참고
# fileinput - 여러 입력 스트림에서 줄을 이터레이트 하기: https://docs.python.org/ko/3/library/fileinput.html
# 파일 읽고 쓰기: https://wikidocs.net/26

In [67]:
import fileinput
import glob

def multi_file_read(*args):
    
    if args: dirname = args[0]
    else: dirname = os.getcwd()

    with fileinput.input(glob.glob(f'{dirname}/*.txt')) as f:
        for line in f:
            print(line)

multi_file_read('./archive')
multi_file_read()

1
2
3
4
1
2
3
4


## 038 디렉터리와 파일을 비교하려면? ― filecmp

In [ ]:
# 038 디렉터리와 파일을 비교하려면? ― filecmp
# -----------------------------------------
# filecmp는 파일 두 개 또는 디렉터리 두 곳을 비교할 때 사용하는 모듈이다.

# 문제
# ----
# 두 디렉터리의 차이를 다음과 같은 조건으로 확인하는 프로그램은 어떻게 작성해야 할까?

# a 디렉터리에만 있고 b 디렉터리에는 없는 파일은 다음과 같이 출력한다.
# a: 파일명

# b 디렉터리에만 있고 a 디렉터리에는 없는 파일은 다음과 같이 출력한다.
# b: 파일명

# a 디렉터리와 b 디렉터리에 모두 있으나 파일의 내용이 서로 다를 때는 다음과 같이 출력한다.
# x: 파일명

In [ ]:
# 풀이
# ----
# filecmp.dircmp()를 사용하면 문제에서 요구하는 프로그램을 쉽게 만들 수 있다.

# [파일명: filecmp_sample.py]

# import filecmp

# fd = filecmp.dircmp('a', 'b')

# for a in fd.left_only:
#     print("a: %s" % a)

# for b in fd.right_only:
#     print("b: %s" % b)

# for x in fd.diff_files:
#     print("x: %s" % x)
# 먼저 fd = filecmp.dircmp('a', 'b')처럼 a, b 디렉터리를 비교하고자 fd 객체를 생성한다. 그리고 a 디렉터리에만 있는 파일은 fd.left_only, b 디렉터리에만 있는 파일은 fd.right_only, 두 곳 모두에 있지만 내용이 다른 파일은 fd.diff_files로 구할 수 있다.

# 참고
# filecmp- 파일과 디렉터리 비교: https://docs.python.org/ko/3/library/filecmp.html

In [68]:
import filecmp

fd = filecmp.dircmp('a', 'b')

for a in fd.left_only:
    print('a: %s' %a)